In [ ]:
!pip install chess

In [1]:
import chess
import random
import math
from copy import deepcopy
from IPython.display import clear_output
from IPython.display import Audio
from time import sleep
from typing import Union
import json
import requests
import re

In [2]:
# Piece-square tables
pieces = {'pawn': 100, 'knight': 320, 'bishop': 330, 'rook': 500, 'queen': 900}
pawn_table = [
    0, 0, 0, 0, 0, 0, 0, 0,
    5, 10, 10, -20, -20, 10, 10, 5,
    5, -5, -10, 0, 0, -10, -5, 5,
    0, 0, 0, 20, 20, 0, 0, 0,
    5, 5, 10, 25, 25, 10, 5, 5,
    10, 10, 20, 30, 30, 20, 10, 10,
    50, 50, 50, 50, 50, 50, 50, 50,
    0, 0, 0, 0, 0, 0, 0, 0]

knight_table = [
    -50, -40, -30, -30, -30, -30, -40, -50,
    -40, -20,   0,   5,   5,   0, -20, -40,
    -30,   5,  10,  15,  15,  10,   5, -30,
    -30,   0,  15,  20,  20,  15,   0, -30,
    -30,   5,  15,  20,  20,  15,   5, -30,
    -30,   0,  10,  15,  15,  10,   0, -30,
    -40, -20,   0,   0,   0,   0, -20, -40,
    -50, -40, -30, -30, -30, -30, -40, -50]

bishop_table = [
    -20, -10, -10, -10, -10, -10, -10, -20,
    -10,   5,   0,   0,   0,   0,   5, -10,
    -10,  10,  10,  10,  10,  10,  10, -10,
    -10,   0,  10,  10,  10,  10,   0, -10,
    -10,   5,   5,  10,  10,   5,   5, -10,
    -10,   0,   5,  10,  10,   5,   0, -10,
    -10,   0,   0,   0,   0,   0,   0, -10,
    -20, -10, -10, -10, -10, -10, -10, -20]

rook_table = [
     0,  0,  0,  5,  5,  0,  0,  0,
    -5,  0,  0,  0,  0,  0,  0, -5,
    -5,  0,  0,  0,  0,  0,  0, -5,
    -5,  0,  0,  0,  0,  0,  0, -5,
    -5,  0,  0,  0,  0,  0,  0, -5,
    -5,  0,  0,  0,  0,  0,  0, -5,
     5, 10, 10, 10, 10, 10, 10,  5,
     0,  0,  0,  0,  0,  0,  0,  0]

queen_table = [
    -20, -10, -10, -5, -5, -10, -10, -20,
    -10,   0,   0,  0,  0,   0,   0, -10,
    -10,   5,   5,  5,  5,   5,   0, -10,
      0,   0,   5,  5,  5,   5,   0,  -5,
     -5,   0,   5,  5,  5,   5,   0,  -5,
    -10,   0,   5,  5,  5,   5,   0, -10,
    -10,   0,   0,  0,  0,   0,   0, -10,
    -20, -10, -10, -5, -5, -10, -10, -20]

king_table = [
     20,  30,  10,   0,   0,  10,  30,  20,
     20,  20,   0,   0,   0,   0,  20,  20,
    -10, -20, -20, -20, -20, -20, -20, -10,
    -20, -30, -30, -40, -40, -30, -30, -20,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30,
    -30, -40, -40, -50, -50, -40, -40, -30]

In [3]:
def make_sleep(time: float):
    '''Sleep for {time} seconds, and print dots'''
    
    time = int(time*10 - 1)
    
    for _ in range(time):
        sleep(0.1)
        print('.', end='')
    print('.')

In [4]:
def get_player_move(board):
    '''Prompt the user for a move in either UCI or SAN notation'''
    while True:
        move = input('Tee käik: ')

        if move == '':
            return move

        try:
            move = board.parse_uci(move)
        except:
            try:
                move = board.parse_san(move)
            except:
                print('Käik on valesti sisestatud või sellist käiku ei saa teha.')
                continue
        
        return move
    
def get_alphabeta_move(board):
    '''Return the best move using the alpha-beta algorithm'''
    
    return alphabeta_move(board, 3)
    
                       
def get_random_move(board):
    '''Return a random move from available legal moves.'''
    
    return random.choice([move for move in board.legal_moves])

In [5]:
def alphabeta_move(board, depth):
    bestMove = chess.Move.null()
    bestRate = -99999
    alpha = -100000
    beta = 100000
    for move in board.legal_moves:
        board.push(move)
        boardRate = -alphabeta(board, -beta, -alpha, depth - 1)
        if boardRate > bestRate:
            bestRate = boardRate
            bestMove = move
        if boardRate > alpha:
            alpha = boardRate
        board.pop()
    return bestMove

def alphabeta(board, alpha, beta, depth):
    bestRate = -9999
    if depth == 0:
        return quiesce(board, alpha, beta)
    for move in board.legal_moves:
        board.push(move)
        rate = -alphabeta(board, -beta, -alpha, depth - 1)
        board.pop()
        if rate >= beta:
            return rate
        if rate > bestRate:
            bestRate = rate
        if rate > alpha:
            alpha = rate
    return bestRate

def quiesce(board, alpha, beta):
    rate = evaluate_board(board)
    if rate >= beta:
        return beta
    if alpha < rate:
        alpha = rate
    
    for move in board.legal_moves:
        if board.is_capture(move):
            board.push(move)
            rate = -quiesce(board, -beta, -alpha)
            board.pop()

            if rate >= beta:
                return beta
            if rate > alpha:
                alpha = rate
    return alpha

In [6]:
def evaluate_board(board):
    if board.is_checkmate():
        if board.turn:
            return -9999
        else:
            return 9999
    if board.is_stalemate():
        return 0
    if board.is_insufficient_material():
        return 0

    white_pawns = len(board.pieces(chess.PAWN, chess.WHITE))
    black_pawns = len(board.pieces(chess.PAWN, chess.BLACK))
    white_knights = len(board.pieces(chess.KNIGHT, chess.WHITE))
    black_knights = len(board.pieces(chess.KNIGHT, chess.BLACK))
    white_bishops = len(board.pieces(chess.BISHOP, chess.WHITE))
    black_bishops = len(board.pieces(chess.BISHOP, chess.BLACK))
    white_rooks = len(board.pieces(chess.ROOK, chess.WHITE))
    black_rooks = len(board.pieces(chess.ROOK, chess.BLACK))
    white_queens = len(board.pieces(chess.QUEEN, chess.WHITE))
    black_queens = len(board.pieces(chess.QUEEN, chess.BLACK))

    piece_score = (pieces['pawn'] * (white_pawns - black_pawns) + pieces['knight'] * (white_knights - black_knights)
        + pieces['bishop'] * (white_bishops - black_bishops) + pieces['rook'] * (white_rooks - black_rooks)
        + pieces['queen'] * (white_queens - black_queens))
    
    pawn_score = sum([pawn_table[i] for i in board.pieces(chess.PAWN, chess.WHITE)])
    pawn_score += sum([-pawn_table[chess.square_mirror(i)] for i in board.pieces(chess.PAWN, chess.BLACK)])

    knight_score = sum([knight_table[i] for i in board.pieces(chess.KNIGHT, chess.WHITE)])
    knight_score += sum([-knight_table[chess.square_mirror(i)] for i in board.pieces(chess.KNIGHT, chess.BLACK)])

    bishop_score = sum([bishop_table[i] for i in board.pieces(chess.BISHOP, chess.WHITE)])
    bishop_score += sum([-bishop_table[chess.square_mirror(i)] for i in board.pieces(chess.BISHOP, chess.BLACK)])

    rook_score = sum([rook_table[i] for i in board.pieces(chess.ROOK, chess.WHITE)])
    rook_score += sum([-rook_table[chess.square_mirror(i)] for i in board.pieces(chess.ROOK, chess.BLACK)])

    queen_score = sum([queen_table[i] for i in board.pieces(chess.QUEEN, chess.WHITE)])
    queen_score += sum([-queen_table[chess.square_mirror(i)] for i in board.pieces(chess.QUEEN, chess.BLACK)])

    king_score = sum([king_table[i] for i in board.pieces(chess.KING, chess.WHITE)])
    king_score += sum([-king_table[chess.square_mirror(i)] for i in board.pieces(chess.KING, chess.BLACK)])

    rate = piece_score + pawn_score + knight_score + bishop_score + rook_score + queen_score + king_score
    
    if board.turn:
        return rate
    else:
        return -rate

## Kommentaari genereerimine

In [7]:
piecesNames = {"P":"Valge ettur", "N":"Valge ratsu", "B":"Valge oda",
          "R":"Valge vanker", "Q":"Valge lipp", "K":"Valge kuningas",
          "p":"Must ettur", "n":"Must ratsu", "b":"Must oda",
          "r":"Must vanker", "q":"Must lipp", "k":"Must kuningas"}
takenPieces = {"P":"Valge etturi", "N":"Valge ratsu", "B":"Valge oda",
          "R":"Valge vankri", "Q":"Valge lipu", "K":"Valge kuninga",
          "p":"Musta etturi", "n":"Musta ratsu", "b":"Musta oda",
          "r":"Musta vankri", "q":"Musta lipu", "k":"Musta kuninga"}


def generateVoiceover(text):
    payload = {'text': text, 'speaker': 'Albert'}
    headers = {"Content-Type": "application/json"};
    response = requests.post("https://api.tartunlp.ai/text-to-speech/v2", data=json.dumps(payload), headers=headers)
    return response.content


def getVoiceover(board):
    try:
        return Audio(data=generateVoiceover(buildComment(board)), autoplay=True)
    except:
        print("Kommentaari sünteesimine ebaõnnestus!")

def resolveOutcomeComment(outcome):
    winner = outcome.winner
    if winner is None:
        return "Mäng jäi viiki."
    elif winner:
        return "Valge võitis mängu."
    else:
        return "Must võitis mängu."

    
def describeMoveComment(board, move):
    piece = board.piece_at(move.to_square)
    result = piecesNames[piece.symbol()]
    result += " käis ruudult "
    result += chess.square_name(move.from_square)
    result += " ruudule "
    result += chess.square_name(move.to_square)
    return result


def describeTakenPiece(board, move):
    result = ""
    board.pop()
    takenPiece = board.piece_at(move.to_square)
    if takenPiece is not None:
        if board.turn:
            result = "Valge võttis ära " + takenPieces[takenPiece.symbol()]
        else:
            result = "Must võttis ära " + takenPieces[takenPiece.symbol()]
    board.push(move)
    return result


def describeCheck(board):
    result = ""
    if board.is_check():
        if board.turn:
            result = "Must annab valgele tuld."
        else:
            result = "Valge annab mustale tuld."
    return result


# MAIN METHOD
# - Assumes the move has already been made
# - Assumes it is still the turn of the player who made the move
def buildComment(board):
    # Variables
    result = ""
    move = board.peek()
    outcome = board.outcome()
    isOngoing = True
    
    # Describe outcome if any
    if (outcome is not None): # game has ended
        result += resolveOutcomeComment(outcome) + " "
        isOngoing = False
    
    # Describe move
    result += describeMoveComment(board, move) + ". "
    
    # Check and describe taken pieces
    result += describeTakenPiece(board, move) + ". "
    
    # Check for check
    if (isOngoing):
        result += describeCheck(board)
    
    return result

## Game loop

In [8]:
move_makers = {'random':get_random_move, 'player':get_player_move, 'ai':get_alphabeta_move}

#todo timing for separate turns
def game(player1, player2, provideCommentary = True):
    get_player1_move = move_makers[player1]
    get_player2_move = move_makers[player2]
    
    board = chess.Board()
    display(board)
    first = True
    
    while True:
        
        # Player 1 move
        p1 = get_player1_move(board)
        if p1 == '':
            break

        board.push(p1)
        #play_move_sound
        
        clear_output()
        display(board)
        if provideCommentary:
            display(getVoiceover(board))
            make_sleep(5)

        if board.is_game_over(): break

        if player1 == 'player':
            make_sleep(2)
        else:
            make_sleep(0.5)
        
        
        
        # Player 2 move
        p2 = get_player2_move(board)
        if p2 == '':
            break
        board.push(p2)
        #play_move_sound
        
        clear_output()
        display(board)
        if provideCommentary:
            display(getVoiceover(board))
            make_sleep(5)

        if board.is_game_over(): break

        if player2 == 'player':
            make_sleep(2)
        else:
            make_sleep(0.5)
        
    return board

# Mäng
Võimalikud mängijad:


*   ai - meie ehitatud AI
*   random - valitakse võimalikest käikudest suvaline
*   player - kasutaja ise

Neid parameetreid võib vahetada "game" funktsiooni väljakutses vastavalt vajadusele. AI oskab nii musta kui ka valgena käia, nii et "ai" võib panna nii player1 kui ka player2 parameetriks.

Käigu sisestus: käike võib sisestada standartses UCI formaadis (nt Nf3 ehk ratsu liigub ruudule f3) või SAN formaadis (nt b2b4 ehk nupp ruudul b2 liigub ruudule b4). Kui soovite mängu lõpetada, siis võite sisestada tühja sõne.



In [ ]:
board = game(player1 = 'ai', player2 = 'player', provideCommentary = True)